In [218]:
from collections import defaultdict, Counter
from tqdm import tqdm
import csv
import os
import pandas as pd
import numpy as np

In [107]:
ROOT = '/mnt/data0/lucy/manosphere/'
ANN_FILE = ROOT + 'data/ann_sig_entities.csv'
COREF_LOGS = '/mnt/data0/dtadimeti/manosphere/logs/'
COREF_REDDIT = COREF_LOGS + 'coref_reddit/'
COREF_FORUMS = COREF_LOGS + 'coref_forums/'
COREF_CONTROL = COREF_LOGS + 'coref_control/'
COREF_RESULTS = ROOT + 'logs/coref_results/'

In [188]:
def load_vocabulary(): 
    words = []
    with open(ANN_FILE, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader: 
            if row['keep'] == 'Y': 
                if row['entity'].lower() == 'she' or row['entity'].lower() == 'he': 
                    continue
                words.append(row['entity'].lower())
    return words

In [222]:
df = pd.read_csv(COREF_RESULTS + 'coref_reddit_df.csv')
cats = df.community.unique()
cats

array(['PUA', 'MRA', 'Incels', 'TRP', 'MGTOW', 'Femcels', 'FDS', nan],
      dtype=object)

Additional errors.

In [223]:
df[df['community'].isnull()]

,year,community,word,fem,masc,they,it,you
57832,2010,NaN,man,0,2,0,0,0
61605,2017,NaN,someone,0,1,0,0,0
79071,2010,NaN,woman,1,0,0,0,0
79798,2016,NaN,man,0,1,0,0,0
79799,2016,NaN,new man,0,1,0,0,0
88271,2010,NaN,guy,0,2,0,0,0
88272,2010,NaN,men,0,0,1,0,0
88994,2009,NaN,mother,1,0,0,0,0
88995,2009,NaN,young children,0,0,1,0,0
88996,2009,NaN,children,0,0,1,0,0


In [224]:
print(sum(df['fem'].to_list()))
print(sum(df['masc'].to_list()))

1104928
818746


In [225]:
1 - (sum(df['fem'].to_list()) / (sum(df['fem'].to_list()) + sum(df['masc'].to_list())))

0.4256157748142356

### Word coverage

In [238]:
totals = df.groupby('word').sum()
totals['total'] = totals['fem'] + totals['masc']
totals = totals[totals['total'] > 20]
totals.sort_values(by=['total'])

,year,fem,masc,they,it,you,total
word,,,,,,,
newb,30241,5,16,2,18,0,21
man child,22190,3,18,0,1,0,21
club owner,26207,0,21,2,2,0,21
mature woman,32244,21,0,1,0,0,21
sisterhood,36297,21,0,28,20,0,21
...,...,...,...,...,...,...,...
wife,100742,80607,690,72,748,541,81297
man,108790,3251,172508,223,275,2234,175759
guy,104763,3183,181414,334,2567,444,184597


We want to get a sense of how many words are gendered. Words that are not gendered likely don't show up often enough to matter much, but there is a long tail that could be important to consider. 

It's possible that we should do a two-step process where we first use markers and then use coref. 

In [239]:
solid_labels = totals.index.to_list()
vocab = load_vocabulary()
missing = set(vocab) - set(solid_labels)
print("NO COREF SIGNAL:", len(missing) / len(vocab))
print("COREF SIGNAL:",len(solid_labels)/ len(vocab))
# TODO: should probably get a better seed set
gendered_words = ['female', 'men', 'females', 'women', 'woman', 
                  'man', 'guy', 'guys', 'boy', 'boys', 'girls', 
                  'girl', 'male', 'males', 'mother', 'mothers', 'wife', 'wives', 
                 'father', 'fathers', 'husband', 'husbands', 'dude', 'chicks', 'chick', 'dudes', 
                  'son', 'sons', 'daughter', 'daughters'] # boyfriend, queen, king
counted = 0
new_missing = set()
for w in missing: 
    tokens = w.split()
    if tokens[-1] in gendered_words: 
        counted += 1
    elif tokens[0] in gendered_words: 
        counted += 1
    else: 
        new_missing.add(w)
print("NO COREF + NO MARKERS:", len(new_missing) / len(vocab))
print(new_missing)

NO COREF SIGNAL: 0.7693238541824232
COREF SIGNAL: 0.23067614581757676
NO COREF + NO MARKERS: 0.5867493571320527
{'cheating whores', 'ugly subhuman', 'religious zealots', '4 kids', 'reasonable people', 'jewish community', 'childrens', 'egalitarians', 'bigots', 'fucking dick', 'successful ones', 'older kids', 'first love', 'complete loser', 'business people', 'adults', 'toastmasters', 'lender', 'happy family', 'goths', 'boogeyman', 'ideologues', 'selectors', 'djs', 'sedditor', 'sjw crowd', 'musicians', 'other humans', 'two others', 'afghans', 'hot bitches', 'matriarch', 'supermodels', 'public figures', 'sexual being', 'instructors', 'oppressed group', 'good therapist', 'victors', 'bisexuals', 'duff', 'hvm', 'directors', 'perfect match', 'shitty ones', 'white folks', 'volcels', 'chadlet', 'tradesman', 'blackcels', 'japanese people', 'dreamers', 'potential rapist', 'co workers', 'fucktoy', 'priests', 'most kids', 'hunters', 'awkward person', 'travellers', 'darling', 'young people', 'normal

### Popular fem words in reddit

In [240]:
def show_top_fem(cat): 
    cat_df = df[df.community == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['fem_frac'] = cat_totals['fem'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_fem = cat_totals[cat_totals['fem_frac'] == 1]
    return cat_fem.sort_values(by=['total'])

In [241]:
for cat in cats: 
    print('------' + cat + '-------')
    print(show_top_fem(cat).head())

------PUA-------
               year  fem  masc  they  it  you  total  fem_frac
word                                                          
twin sister   12082   11     0     0   0    0     11       1.0
chinese girl  14105   11     0     0   0    0     11       1.0
swedish girl  12085   11     0     0   0    0     11       1.0
better girl   12088   11     0     0   1    0     11       1.0
1 woman       14108   12     0     0   0    0     12       1.0
------MRA-------
                 year  fem  masc  they  it  you  total  fem_frac
word                                                            
happy wife      14106   11     0     0   0    0     11       1.0
next girl       14110   11     0     0   0    0     11       1.0
average girl     8055   11     0     0   0    0     11       1.0
normal woman    16114   11     0     0   0    0     11       1.0
straight woman  12084   11     0     0   0    0     11       1.0
------Incels-------
                  year  fem  masc  they  it  you  

TypeError: can only concatenate str (not "float") to str

### Popular masc words in reddit

In [242]:
def show_top_masc(cat): 
    cat_df = df[df.community == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['masc_frac'] = cat_totals['masc'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_masc = cat_totals[cat_totals['masc_frac'] == 1]
    return cat_masc.sort_values(by=['total'])

In [243]:
for cat in cats: 
    print('------' + cat + '-------')
    print(show_top_masc(cat).head())

------PUA-------
              year  fem  masc  they  it  you  total  masc_frac
word                                                          
rich man     14103    0    11     0   0    1     11        1.0
best buddy    8055    0    11     0   0    0     11        1.0
nerdy guy    16123    0    11     0   0    0     11        1.0
big guy      12090    0    11     0   1    0     11        1.0
billionaire  14102    0    11     3   4    0     11        1.0
------MRA-------
            year  fem  masc  they  it  you  total  masc_frac
word                                                        
tall man   10074    0    11     0   0    0     11        1.0
white kid  12089    0    11     0   0    0     11        1.0
wise man   10074    0    11     0   0    0     11        1.0
fat guy    10070    0    11     0   2    0     11        1.0
blind man  10072    0    11     0   0    0     11        1.0
------Incels-------
             year  fem  masc  they  it  you  total  masc_frac
word            

TypeError: can only concatenate str (not "float") to str

### Popular neut words in reddit

In [138]:
def show_top_neut(cat): 
    cat_df = df[df.community == cat]
    cat_totals = cat_df.groupby('word').sum()
    cat_totals['total'] = cat_totals['fem'] + cat_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    cat_totals = cat_totals[cat_totals['total'] > 10] 
    cat_totals['fem_frac'] = cat_totals['fem'] / (cat_totals['fem'] + cat_totals['masc'])
    cat_neut = cat_totals[cat_totals.fem_frac.between(0.48, 0.52)]
    return cat_neut.sort_values(by=['total'], ascending = False)

In [169]:
for cat in cats: 
    print('------' + cat + '-------')
    print(show_top_neut(cat).head())

------PUA-------
               fem  masc  neut  it  you  total  fem_frac
word                                                    
bro            126   129     0  34   21    255  0.494118
drunk friend    16    15     0   0    0     31  0.516129
clerk           12    12     0   5    0     24  0.500000
moron            6     6     0   1    0     12  0.500000
new boyfriend    6     6     0   1    0     12  0.500000
------MRA-------
            fem  masc  neut    it  you  total  fem_frac
word                                                   
victim     1345  1448     0  1166    2   2793  0.481561
author     1329  1415     0   305    1   2744  0.484329
professor   304   314     0    24    1    618  0.491909
cop         113   114     0   223    0    227  0.497797
abuser       89    84     0    28    0    173  0.514451
------Incels-------
            fem  masc  neut   it  you  total  fem_frac
word                                                  
teacher     254   239     0    0    1    493 

TypeError: can only concatenate str (not "float") to str

### Words with "it" 

In [174]:
df.sort_values(by=['it'], ascending = False).head(15)

,month,community,word,fem,masc,neut,it,you
164956,2018-05,Incels,incel,19,108,0,114,0
16802,2018-04,Incels,incel,12,120,0,98,0
164828,2018-05,Incels,incels,13,10,0,98,0
16720,2018-04,Incels,incels,7,15,0,84,0
228451,2018-07,Incels,incel,11,99,0,83,0
146995,2018-08,Incels,incel,14,89,0,82,0
261759,2018-06,Incels,incel,25,112,0,81,0
327476,2018-03,Incels,incels,4,15,0,77,1
261810,2018-06,Incels,incels,9,10,0,76,0
11053,2012-11,MRA,child,37,94,0,73,0


### Gender over time 

In [232]:
# These are the words whose % fem ranges over time are the largest,
# maybe top three words with biggest % range in each community, e.g. “cat	10% - 50%”, only calculate fraction 
# if there are more than 10 occurrences in each community and month. 

def show_top_fem_range(cat):
    # filtering for the argument category
    cat_df = df[df.community == cat]
    totals = cat_df.groupby(['year', 'word'], as_index = False).sum()
    totals['total'] = totals['fem'] + totals['masc']
    totals = totals[totals['total'] > 10]
    totals['fem_frac'] = totals['fem'] / (totals['fem'] + totals['masc'])
    
    # filter for words that show up in more than 1 of the months/time periods 
    # (initially picked 85 to be more than half of all the months but idk if needed)
    is_multi = totals["word"].value_counts() > 1
    filtered = totals[totals["word"].isin(is_multi[is_multi].index)]
    
    # get the max and min fem_frac for each word
    word_keys = filtered['word'].unique().tolist()
    max_fems = []
    min_fems = []
    max_months = []
    min_months = []
    for word in word_keys: 
        df_subset = filtered[filtered['word'] == word]
        max_fem = df_subset['fem_frac'].max()
        min_fem = df_subset['fem_frac'].min()
        max_month = df_subset[df_subset['fem_frac'] == max_fem]['year'].max()
        min_month = df_subset[df_subset['fem_frac'] == min_fem]['year'].min()
        
        max_fems.append(max_fem)
        min_fems.append(min_fem)
        max_months.append(max_month)
        min_months.append(min_month)
    
    
    d = {'word': [], 'min month': [], 'min': [], 'max month': [], 'max':[], 'diff': []}
    for i in range(len(word_keys)):
        d['word'].append(word_keys[i])
        d['min month'].append(min_months[i])
        d['min'].append(min_fems[i])
        d['max month'].append(max_months[i])
        d['max'].append(max_fems[i])
        d['diff'].append(max_fems[i] - min_fems[i])
    
    diffs = pd.DataFrame(data=d)
    return diffs.sort_values(by = ['diff'], ascending = False)


In [237]:
# words in each month in Incels that appear more than 10 times in that month
for cat in cats: 
    print('------' + cat + '-------')
    print(show_top_fem_range(cat).head())

------PUA-------
              word  min month       min  max month       max      diff
74     good friend       2015  0.181818       2019  0.666667  0.484848
90      one friend       2012  0.200000       2011  0.666667  0.466667
120           baby       2012  0.545455       2019  1.000000  0.454545
138   other person       2013  0.222222       2017  0.666667  0.444444
29   mutual friend       2019  0.500000       2010  0.937500  0.437500
------MRA-------
          word  min month       min  max month       max      diff
126     anyone       2012  0.045455       2017  0.642857  0.597403
141       cops       2016  0.347826       2013  0.857143  0.509317
216  bartender       2017  0.250000       2019  0.750000  0.500000
44         cop       2010  0.272727       2018  0.750000  0.477273
220  detective       2018  0.000000       2019  0.470588  0.470588
------Incels-------
           word  min month       min  max month       max      diff
13   girlfriend       2019  0.094544       2013  0

TypeError: can only concatenate str (not "float") to str

### Popular words in control

In [161]:
control_df = pd.read_csv('pronoun_control_df.csv')

def show_top_fem_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear more than 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['fem_frac'] = control_totals['fem'] / (control_totals['fem'] + control_totals['masc'])
    control_fem = control_totals[control_totals['fem_frac'] == 1]
    return control_fem.sort_values(by=['total'], ascending = False)

def show_top_masc_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['masc_frac'] = control_totals['masc'] / (control_totals['fem'] + control_totals['masc'])
    control_masc = control_totals[control_totals['masc_frac'] == 1]
    return control_masc.sort_values(by=['total'], ascending = False)

def show_top_neut_control(): 
    control_totals = control_df.groupby('word').sum()
    control_totals['total'] = control_totals['fem'] + control_totals['masc'] 
    # filter to only those that appear at least 10 times as she or he
    control_totals = control_totals[control_totals['total'] > 10] 
    control_totals['fem_frac'] = control_totals['fem'] / (control_totals['fem'] + control_totals['masc'])
    control_neut = control_totals[control_totals['fem_frac'].between(0.45, 0.55)]
    return control_neut.sort_values(by=['total'], ascending = False)

FileNotFoundError: [Errno 2] File b'pronoun_control_df.csv' does not exist: b'pronoun_control_df.csv'

In [ ]:
show_top_neut_control()

### Gender differences

In [54]:
# len(df.word.unique()
df = df.groupby('word').sum()
df['total'] = df['fem'] + df['masc']
df = df[df['total'] > 10] 
df['fem_frac'] = df['fem'] / (df['fem'] + df['masc'])
df = df.sort_values(by=['fem'], ascending = False)
df


,fem,masc,neut,total,fem_frac
word,,,,,
girl,308205,2567,226,310772,0.991740
woman,248514,1024,838,249538,0.995896
wife,74069,576,55,74645,0.992283
mother,31271,275,139,31546,0.991283
mom,23830,893,186,24723,0.963880
...,...,...,...,...,...
physicist,0,29,1,29,0.000000
tall man,0,59,1,59,0.000000
soccer player,0,11,0,11,0.000000


In [56]:
control_df = control_df.groupby('word').sum()
control_df['total'] = control_df['fem'] + control_df['masc']
control_df = control_df[control_df['total'] > 10] 
control_df['fem_frac'] = control_df['fem'] / (control_df['fem'] + control_df['masc'])
control_df = control_df.sort_values(by=['fem'], ascending = False)
control_df


,fem,masc,neut,total,fem_frac
word,,,,,
mom,22693,764,200,23457,0.967430
wife,22389,165,18,22554,0.992684
girl,22147,269,29,22416,0.988000
woman,17238,131,56,17369,0.992458
mother,13385,107,53,13492,0.992069
...,...,...,...,...,...
composer,0,29,7,29,0.000000
common man,0,34,1,34,0.000000
colonel,0,25,0,25,0.000000


In [57]:
merged_df = df.merge(control_df, how='inner', left_index=True, right_index=True)
merged_df

,fem_x,masc_x,neut_x,total_x,fem_frac_x,fem_y,masc_y,neut_y,total_y,fem_frac_y
word,,,,,,,,,,
girl,308205,2567,226,310772,0.991740,22147,269,29,22416,0.988000
woman,248514,1024,838,249538,0.995896,17238,131,56,17369,0.992458
wife,74069,576,55,74645,0.992283,22389,165,18,22554,0.992684
mother,31271,275,139,31546,0.991283,13385,107,53,13492,0.992069
mom,23830,893,186,24723,0.963880,22693,764,200,23457,0.967430
...,...,...,...,...,...,...,...,...,...,...
layman,0,11,0,11,0.000000,1,11,0,12,0.083333
first man,0,69,0,69,0.000000,0,31,0,31,0.000000
physicist,0,29,1,29,0.000000,1,22,2,23,0.043478


In [59]:
merged_df['difference'] = (merged_df['fem_frac_x'] - merged_df['fem_frac_y']).abs()
merged_df = merged_df.sort_values(by=['difference'], ascending = False)

In [61]:
merged_df

,fem_x,masc_x,neut_x,total_x,fem_frac_x,fem_y,masc_y,neut_y,total_y,fem_frac_y,difference
word,,,,,,,,,,,
brat,21,5,1,26,0.807692,2,10,1,12,0.166667,0.641026
expert,68,35,25,103,0.660194,8,48,31,56,0.142857,0.517337
band,15,2,382,17,0.882353,8,13,1799,21,0.380952,0.501401
cunt,789,205,60,994,0.793763,35,77,12,112,0.312500,0.481263
sitter,11,3,1,14,0.785714,7,14,0,21,0.333333,0.452381
...,...,...,...,...,...,...,...,...,...,...,...
trans woman,131,0,1,131,1.000000,73,0,4,73,1.000000,0.000000
grown woman,120,0,5,120,1.000000,32,0,0,32,1.000000,0.000000
own daughter,112,0,0,112,1.000000,48,0,0,48,1.000000,0.000000


### Pronoun sparsity

In [84]:
df = pd.read_csv('pronoun_df.csv')
# df = df.groupby('word').sum()
# df.shape[0]

# total vocab words that show up in reddit with masc/fem/neut pronouns is 6373

df_totals = df.groupby('word').sum()
# df_totals['total'] = df_totals['fem'] + df_totals['masc'] 
df_totals['total'] = df_totals['fem'] + df_totals['masc'] + df_totals['neut']

df_totals['neut_frac'] = df_totals['neut'] / (df_totals['fem'] + df_totals['masc'] + df_totals['neut'])
df_totals 

df_neut = df_totals[df_totals['neut_frac'] >= 0.5].sort_values(by = ['neut_frac'], ascending = False)
# df_neut.head(20)
df_neut

# df_sparse = df_totals[df_totals['total'] <= 10]
# df_sparse



# 4300 words have less than 10 occurrences with masc or fem pronouns => ~70%
# 1438 words have less than 10 occurrences with masc or fem or neut pronouns => ~23%

# 3716 words are mostly "they" words => ~60%



,fem,masc,neut,total,neut_frac
word,,,,,
zombies,0,0,97,97,1.0
monarchs,0,0,18,18,1.0
mobile users,0,0,1,1,1.0
mockingbird,0,0,1,1,1.0
moderate feminists,0,0,86,86,1.0
...,...,...,...,...,...
walking wallet,2,0,2,4,0.5
total loser,0,2,2,4,0.5
changs,0,2,2,4,0.5


### Evaluation 

Comparing booknlp coref vs. spacy coref

In [212]:
gold_masc = set()
gold_fem = set()
with open(ROOT + 'logs/gender_gold_labels.csv', 'r') as infile: 
    reader = csv.DictReader(infile)
    for row in reader: 
        if row['gendered?'] == 'm':
            gold_masc.add(row['word (singular)'].lower())
        if row['gendered?'] == 'f': 
            gold_fem.add(row['word (singular)'].lower())

In [213]:
david_labels = Counter()
with open(ROOT + 'logs/temp_gender.txt', 'r') as infile: 
    reader = csv.DictReader(infile, delimiter='\t')
    for row in reader: 
        if row['proper'] != 'nom': continue
        if (float(row['he/him/his']) + float(row['she/her'])) < 3: continue
        david_labels[row['term']] = float(row['she/her']) / (float(row['he/him/his']) + float(row['she/her']))

In [214]:
df = pd.read_csv(COREF_RESULTS + 'coref_reddit_df.csv')

In [215]:
df = df.groupby('word').sum()
df['fem_frac'] = df['fem'] / (df['fem'] + df['masc'])
df = df[['fem_frac']].dropna()
df = df.to_dict()

In [216]:
spacy_labels = df['fem_frac']

In [219]:
# average score for m words
spacy_scores = []
david_scores = []
for w in gold_masc: 
    if w in spacy_labels and w in david_labels: 
        spacy_scores.append(spacy_labels[w])
        david_scores.append(david_labels[w])
print("masc words")
print("SPACY:", np.mean(spacy_scores), "BOOKNLP:", np.mean(david_scores))

masc words
SPACY: 0.20069466570475966 BOOKNLP: 0.31480068170482933


In [221]:
# average score for f words
spacy_scores = []
david_scores = []
for w in gold_fem: 
    if w in spacy_labels and w in david_labels: 
        spacy_scores.append(spacy_labels[w])
        david_scores.append(david_labels[w])
print("fem words")
print("SPACY:", np.mean(spacy_scores), "BOOKNLP:", np.mean(david_scores))

fem words
SPACY: 0.8515987971507715 BOOKNLP: 0.7529589257733309
